In [1]:
import chess
import chess.engine

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [2]:
from generate_data import *

In [3]:
class ChessDataset(Dataset):
    """Chess Positions Evaluation Dataset"""

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        position = np.array(self.df.iloc[idx, :-1])
        eval_ = np.array(self.df.iloc[idx, -1]/100)
        
        sample = {'position': torch.from_numpy(position), 'eval': torch.from_numpy(eval_)}

        return sample

In [4]:
dataset = ChessDataset(csv_file='data/stockfish_depth0d.csv')

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = nn.Conv2d(8, 8, 2) #input, output channels, kernel_size
        self.conv2 = nn.Conv2d(8, 8, 3)
        self.conv3 = nn.Conv2d(8, 8, 3)
        self.conv4 = nn.Conv2d(8, 8, 3)
        self.fc1 = nn.Linear(8, 64) 
        self.fc2 = nn.Linear(64, 1) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)   
        x = torch.reshape(x, (-1,)) # flattens into 1d tensor
        x = self.fc1(F.relu(x))
        x = self.fc2(torch.sigmoid(x))
        #x = self.fc1(x)
        #x = self.fc2(x)

        return x

In [6]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)

net = Net()#.cuda()

In [7]:
t = torch.tensor([[1., -1.], [1., -1.]])
t = t.cuda()
t.get_device()

0

# TRAINING DATA

In [8]:
#torch.cuda.get_device_name(device)

In [9]:
trainloader = DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=0) 
#num_workers = 0 doesn't use GPU. GPU seems to be a lot slower than CPU,.
#data set is prob too big or my laptop nvidia GPU is just trash

optimizer = optim.Adam(net.parameters(), lr = 0.0009) #lr = 0.001
criterion = nn.MSELoss() 

for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['position'].reshape(1, 8, 8, 8).float()#.cuda()
        labels = data['eval'].float()#.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 9.327
[1,   400] loss: 8.757
[1,   600] loss: 10.464
[1,   800] loss: 9.096
[1,  1000] loss: 8.089
[1,  1200] loss: 9.213
[1,  1400] loss: 8.982
[1,  1600] loss: 6.889
[1,  1800] loss: 8.981
[1,  2000] loss: 7.149
[1,  2200] loss: 7.547
[1,  2400] loss: 7.547
[1,  2600] loss: 8.887
[1,  2800] loss: 8.503
[1,  3000] loss: 6.523
[1,  3200] loss: 6.586
[1,  3400] loss: 7.185
[1,  3600] loss: 6.734
[1,  3800] loss: 7.286
[1,  4000] loss: 7.096
[1,  4200] loss: 5.786
[1,  4400] loss: 6.180
[1,  4600] loss: 6.817
[1,  4800] loss: 7.275
[1,  5000] loss: 5.937
[1,  5200] loss: 5.779
[1,  5400] loss: 7.625
[1,  5600] loss: 5.382
[1,  5800] loss: 6.451
[1,  6000] loss: 7.342
[1,  6200] loss: 4.528
[1,  6400] loss: 6.500
[1,  6600] loss: 6.505
[1,  6800] loss: 7.358
[1,  7000] loss: 5.300
[1,  7200] loss: 6.030
[1,  7400] loss: 5.769
[1,  7600] loss: 5.260
[1,  7800] loss: 5.479
[1,  8000] loss: 6.994
[1,  8200] loss: 5.308
[1,  8400] loss: 6.351
[1,  8600] loss: 7.799
[1,  8800]

In [10]:
def compare_eval():
    board = generate_rand_board()
    torch_board = torch.from_numpy(convert_board(board).reshape(1, 8, 8, 8)).float()
    net_eval = net(torch_board)
    sf_eval = stockfish_evaluation(board)
    print(f"NN: {net_eval} Stockfish: {sf_eval/100}")
    

In [11]:
for _ in range(5):
    compare_eval()

NN: tensor([-7.9722], grad_fn=<AddBackward0>) Stockfish: -9.78
NN: tensor([1.4270], grad_fn=<AddBackward0>) Stockfish: -0.52
NN: tensor([-0.4037], grad_fn=<AddBackward0>) Stockfish: -1.12
NN: tensor([10.1874], grad_fn=<AddBackward0>) Stockfish: 9.32
NN: tensor([0.1497], grad_fn=<AddBackward0>) Stockfish: 0.0
